In [18]:
!pip install huggingface_hub -q

In [19]:
from huggingface_hub import login
login()

In [20]:
!pip install transformers sentencepiece sacrebleu -q

In [49]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("NLLB model loaded successfully")

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


NLLB model loaded successfully


In [51]:
def translate_en_to_ta(text):
    # Set source language
    tokenizer.src_lang = "eng_Latn"

    inputs = tokenizer(text, return_tensors="pt")

    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("tam_Taml"),
        max_length=200
    )

    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [52]:
sample = "The patient has chest pain and shortness of breath."

result = translate_en_to_ta(sample)

print("English:", sample)
print("Tamil:", result)

English: The patient has chest pain and shortness of breath.
Tamil: நோயாளிக்கு மார்பக வலி மற்றும் மூச்சுத் திணறல் உள்ளது.


In [53]:
def healthcare_pipeline(text):
    tamil_translation = translate_en_to_ta(text)

    # Example placeholder for classifier
    # prediction = classify_with_biobert(text)

    return {
        "original_text": text,
        "tamil_translation": tamil_translation,
        # "prediction": prediction
    }

In [54]:
print(healthcare_pipeline("The patient has chest pain."))

{'original_text': 'The patient has chest pain.', 'tamil_translation': 'நோயாளிக்கு மார்பக வலி உள்ளது.'}


In [55]:
model.save_pretrained("translator_model")
tokenizer.save_pretrained("translator_model")

print("Translator model saved")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Translator model saved


In [58]:
import shutil
from google.colab import files

shutil.make_archive("translator_model", 'zip', "translator_model")
files.download("translator_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>